# Section 3. Tabular Data Exploration

#### Instructor: Pierre Biscaye

The content of this notebook draws on material from Gabor's Data Analysis in Python [course](https://github.com/gabors-data-analysis/da-coding-python).
    
This notebook covers some additional basics of working with tabular data. 
It does not go into details of regression analysis or other econometrics as that is out of scope for the course. 
We will cover more details of regression analysis in a later notebook as we prepare to get into machine learning.  
    
### Sections
    
1. Descriptive statistics for data frames                       
2. Checking for outliers
3. Hypothesis testing
4. Associations between variables
   
### Libraries loaded
* pandas
* matplotlib.pyplot
* numpy
* seaborn
* scipy
* statsmodels.formula.api

### Files loaded
* gapminder.csv   

### Let's start by importing packages we know we'll need

Others will be introduced later.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Import Data

We'll be using data from the billion price project. The Billion Prices [project](http://www.thebillionpricesproject.com/) collects online and offline prices of selected products sold by selected retailers. It was founded in 2008 by Alberto Cavallo and Roberto Rigobon and aims to collect price information all over the world. 

The data are available as a CSV online, so we can practice loading that directly into a data frame. 

*Note*: the encoding ensures special characters are read in correctly. Different encodings may be needed for different text files.

In [ ]:
bpp_original = pd.read_csv("https://osf.io/yhbr5/download", encoding="latin-1")


In [ ]:
# Check variables
bpp_original.columns


Let's **create a new variable**, the price difference between the online and offline price, using vectorization.

In [ ]:
bpp_original["p_diff"] = bpp_original["price_online"] - bpp_original["price"]


## 1. Descriptive statistics

First, check all the variables in DataFrame using the built-in summary statistics method `describe()`. Note that this only applies to numeric variables.

In [ ]:
bpp_original.describe()


Compare key variables

In [ ]:
bpp_original.filter(["price", "price_online", "p_diff"]).describe()


In [ ]:
# Suppress scientific notation in Pandas
pd.options.display.float_format = '{:.2f}'.format
bpp_original.filter(["price", "price_online", "p_diff"]).describe()

We can also request specific percentiles in the summary stats.

In [ ]:
bpp_original.filter(["price", "price_online", "p_diff"]).describe(percentiles=[0.25, 0.5, 0.75, 0.95])

In [ ]:
# Every 10th percentile
bpp_original.filter(["price", "price_online", "p_diff"]).describe(percentiles = np.arange(0.1, 1, 0.1))

Put the descriptives into columns and variables into rows for a nicer looking table.

In [ ]:
sumstats = bpp_original.filter(["price", "price_online", "p_diff"]).describe().transpose()
sumstats

Let's export this table as a csv!

In [ ]:
sumstats.to_csv('bpp_sumstats.csv', index=True)

### Descriptive statistics by grouping variable.

Let's look at price differences by country.

For this, you need to group the data and apply the required statistics to the appropriate columns

In [ ]:
bpp_original.groupby("COUNTRY").agg(
    mean_price_diff=("p_diff", "mean"), median_price_diff=("p_diff", "median")
)


Let's create a new function calculating the range of values and add this to the descriptive statistics.

In [ ]:
def range_function(x):
    return x.max() - x.min()

bpp_original.groupby("COUNTRY").agg(
    mean_price_diff=("p_diff", "mean"),
    median_price_diff=("p_diff", "median"),
    range_price_diff=("p_diff", range_function),
)


**Practice:** Load the gapminder.csv dataset. Aggregate the data by country, taking the max over years for lifeExp and gdpPercap. Print a table of descriptive statistics for this aggregated dataset, including the 95th percentile in addition to the default statistics. Pivot the table to show statistics as columns, and save it to your working directory.

In [ ]:
gap = pd.read_csv('Data/gapminder.csv')
gap.columns

In [ ]:
# Code here

## 2. Checking for outliers

Before digging deep into analysis, it is useful to visualize the distribution of variables. This is particularly useful for checking for outliers which can unduly affect any analysis.

Let's use the built in pandas `hist()` function.

In [ ]:
bpp_original['price'].hist(bins=30); # remember that adding ; at the end clears a display of the type of output

It is clear: we need to filter out some outlier data!

In [ ]:
# check out the max, compared to p75!
bpp_original.filter(["price"]).describe().transpose()

Let's drop obvious **outliers** for price. 

One possibility is just eyeballing based on the histogram and choosing a round cutoff as a threshold for identifying outliers, such as $1000.

Another possibility is taking a more statistical approach and identifying a percentile cutoff, such as the 95th or 99th percentile. 

After identifying a threshold for outliers, we have to decide how to deal with them. We could:
1. Drop all observations of price above that threshold
2. Set them to missing (which effectively drops them from any analysis)
3. Replace them with other values.

Replacing outliers with another value is called **imputation**. A common approach is to **winsorize** extreme values by replacing them with the median or the value at some cutoff point. It is a good idea to check the sensitivity of your analyses to how you deal with outliers.

For now, let's drop rows where price is above the 95th percentile.

In [ ]:
# Calculate the 95th percentile of the price column
percentile_95 = bpp_original['price'].quantile(0.95)
print(percentile_95)

# Drop rows where price is above the 95th percentile
bpp = bpp_original[bpp_original['price'] <= percentile_95]

print(bpp.shape)
print(bpp_original.shape)


In [ ]:
bpp.filter(["price", "price_online", "p_diff"]).describe()

In [ ]:
bpp['price'].hist()

In [ ]:
#how does the number of bins affect the usefulness of the plot?
bpp['price'].hist(bins=5)

In [ ]:
bpp['price'].hist(bins=25)

#### Kernel density

Histogram or kernel density? A kernel density line is an approximation of the probability density function and is another useful way of plotting the distribution of a variable.
 
We can plot kernel densities using the `seaborn` library.

In [ ]:
import seaborn as sns

In [ ]:
# Create a histogram with a KDE curve
sns.histplot(bpp['price'], kde=True, bins=25, color='blue', alpha=0.5)
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.title('Histogram and KDE for Price')
plt.show()

In [ ]:
# Now for price differences
sns.histplot(bpp['p_diff'], kde=True, bins=25, color='blue', alpha=0.5)
plt.xlabel('Price difference')
plt.ylabel('Frequency')
plt.title('Histogram and KDE for Price difference')
plt.show()

We still have some large outliers! Let's **trim** the  dataset to drop observations where the online price is above the 95th percentile in the original data.

In [ ]:
# Calculate the 95th percentile of the price column
percentile_95 = bpp_original['price_online'].quantile(0.95)
print(percentile_95)
print(bpp.shape)

# Drop rows where price is above the 99th percentile
bpp = bpp[bpp['price_online'] <= percentile_95]
print(bpp.shape)


In [ ]:
bpp.filter(["price", "price_online", "p_diff"]).describe()

In [ ]:
# Now for price differences
sns.histplot(bpp['p_diff'], kde=True, bins=25, color='blue', alpha=0.5)
plt.xlabel('Price difference')
plt.ylabel('Frequency')
plt.title('Histogram and KDE for Price difference')
plt.show()

**Practice:** Plot histograms of population and life expectancy in the gapminder dataset. Do there appear to be any outliers? Drop observations above the 99th percentile for any variable with large positive outliers, and plot the distribution again.

In [ ]:
# Code here

## 3. Hypothesis testing 

All econometric analysis revolves around testing hypotheses about values in the data.

Let's start with a simple hypothesis tests about whether there is any difference between online and offline prices for the goods in this sample.

Test 1: 

H0: the average price difference between price_online - price = 0 \
HA: the average price diff is non-0.

We will use the `stats` package from the `scipy` library for this analysis.

In [ ]:
from scipy import stats

In [ ]:
stats.ttest_1samp(bpp["p_diff"], 0)

*Question*: What do we conclude?

Let us create multiple hypothesis tests to check the hypothesis that online prices are the same as offline for each country!

In [ ]:
testing = bpp.groupby("COUNTRY").agg(
    mean_pdiff=("p_diff", "mean"),
    se_pdiff=("p_diff", "sem"),
    num_obs=("p_diff", "count"),
)
testing


With these statistics we can calculate t-stats for the null hypothesis that the difference is 0 in each country.

In [ ]:
testing["t_stat"] = testing["mean_pdiff"] / testing["se_pdiff"]
testing["p_val"] = stats.t.sf(abs(testing["t_stat"]), df=testing["num_obs"] - 1).round(4)
testing


*Econometrics review*: Interpret the results for each country.

What are the possible dangers of multiple hypothesis testing?

## 4. Association

We are generally most interested in analyzing the relationship between two or more variables.

A good way to start is by plotting variables against each other.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(bpp['price_online'], bpp['price'], alpha=0.6)
plt.show()

If all the points were on the 45 degree line, that would indicate that two values are perfectly equal.

Let's add a 45 degree line.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(bpp['price_online'], bpp['price'], alpha=0.6)
plt.axline([0, 0], [1, 1], color='black')
plt.show()

We can also estimate the linear relationship between the two variables using a linear regression, and add that to the plot.

Let's use the `linregress` function from the `scipy` `stats` package.

In [ ]:
stats.linregress(bpp['price_online'], bpp['price'])

In [ ]:
# We can extract individual elements
slope=stats.linregress(bpp['price_online'], bpp['price']).slope
intc=stats.linregress(bpp['price_online'], bpp['price']).intercept
slope

Let's use these results to add the regression line to the plot.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(bpp['price_online'], bpp['price'], alpha=0.6)
plt.axline([0, 0], [1, 1], color='black', label='45 degree line')
plt.axline(xy1=(0,intc), slope=slope, color='red', label='Linear regression') # plot line with a point and a slop
plt.ylabel('Price', fontsize=12)
plt.xlabel('Price Online', fontsize=12)
plt.title('Association between price online vs offline', fontsize=14)
plt.legend(fontsize=10)
plt.show()

*Question*: What do you conclude from this analysis?

### Running regressions

There are many packages for running regressions in python. Another common one is the statsmodels formula api from the `statsmodels` library. This function allows you to specify the regression equation as a string, and produces output tables similar to what you might see in Stata or R.

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
simple_reg = smf.ols("price ~ price_online", data=bpp).fit()
print(simple_reg.summary())

Simple model, with heteroskedastic robust SE:

In [ ]:
heterorob_reg = smf.ols("price ~ price_online", data=bpp).fit(cov_type="HC3")
print(simple_reg.summary())

We can easily add controls in this package, but adding things like fixed effects requires using different packages. 

You will have to do some searching to figure out what works best for your needs!

This is not an econometrics class so we will not go any further into regressions for now, but there are great resources available if you want to do regression analysis in python.

In [ ]:
multiv_reg = smf.ols("price ~ price_online + year", data=bpp).fit()
print(multiv_reg.summary())